Please note this just example usage of Model Registry Python code, not desciption of all the Python framework that Model Registry offers. 

## Installation
Before we can do any work with Model Registry make sure that Model registry is installed and working. The very first thing we need to do we need load the Model Registry python library 

In [ ]:
%pip install --pre model-registry
%pip install boto3
%run example-utils.py

## Connect to OpenShift
Connect to the Openshift cluster capture some key information about access and Model Registry URLs

In [ ]:

!oc login <host:8443> --username=<myuser> --password=<mypass>  # login into the openshift server

!oc whoami -t # TOKEN
!oc whoami   # USER 

# Grab registry's URL (change the name of registry accordingly in below command)
#HOST
!echo "https://$(oc get routes -n istio-system -l app.kubernetes.io/name=modelregistry-public -o json | jq '.items[].status.ingress[].host | select(contains("-rest"))')" | tr -d '"'

## Set up Access to S3 for storage
Configure the following for storing the models in S3 bucket. Currently Model Registry does not have any storage built into it, it is a index/metadata store only. providing default storage from Model Registry is on the roadmap in future versions.

In [ ]:
# lets download the a model from hugging face and store in S3 bucket
# Edit the below few varibales to modify for your usecase

aws_access_key_id="<YOUR_ACCESS_KEY>"
aws_secret_access_key="<YOUR_SECRET_KEY>"
aws_s3_endpoint="s3.us-east-1.amazonaws.com"
region_name='us-east-1'

file_url = "https://huggingface.co/tarilabs/mnist/resolve/v20231206163028/mnist.onnx"  # Replace with actual file URL
local_file_name = "mnist.onnx"      # Path to save the downloaded file locally

# S3 bucket information
bucket_name = "rareddy-examples"    # Replace with your S3 bucket name
s3_file_name = "models/mnist.onnx"  # Name of the file to save in S3

In [ ]:
import requests
import boto3
from botocore.exceptions import NoCredentialsError

# connect to the S3
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

# Download the file from the web
local_file = download_file(file_url, local_file_name)

# Upload the downloaded file to S3
s3.upload_file(local_file, bucket_name, s3_file_name)
print(f"File uploaded to S3 successfully: {s3_file_name}")

## Import and Connect to the Model Registry
The next is to import the Python library and connect to Model Registry before you can "register" a model in Model Registry. Note that there can be more than one Model Registry in given Open Data Hub cluster with varying degrees of access. You may or may not have access to all the Model Registries that are available. Some could be public registries that are available for all the users and some may be available for certain business units or teams. Your access to given registry is controlled by permissions that are granted to you. If you see any access permissions please check your permissions.

First step is to find Model Registry URL to connect to, use below to find the URL

## Connecting to Model Registry
You can connect to a secure Model Registry using the default constructor (recommended):

In [ ]:
from model_registry import ModelRegistry
import os

# connect to the Model Registry
#registry = ModelRegistry("<MR-HOST>", author="<USER>", user_token="<TOKEN>")

registry = ModelRegistry("https://modelregistry-public-rest.apps.rosa.rareddy-rosa.bd9q.p3.openshiftapps.com", author="admin", user_token="sha256~u2o60sVNjPGC9SJUfgCZXfhoHd3Be7rj62sKnH3BKVI")

# Registering models
Use the following code to register your model in the Model Registry. Once registered, the model can be shared across teams and deployed for inference.

When a model is stored in the Model Registry, it is assigned a name along with a version number that corresponds to the specific instance of the model. If you make updates or modifications to the model and create a new version, you can simply register this new version under the same model name.

To uniquely identify each version of the model, it’s common for developers to use a unique identifier, such as a commit ID, that reflects the changes made in that version. The metadata can be any arbitary data that can help describe the model  

In [ ]:
from model_registry import utils

model = registry.register_model(
    name="mnist",                          # model name
    uri=utils.s3_uri_from(endpoint=aws_s3_endpoint, bucket=bucket_name, region=region_name, path=s3_file_name), # where the model is stored in the S3 and bucket
    version="1.0.0",                       # model version
    description="This is test model for demonstration purposes", # description of the model
    model_format_name="onnx",              # model format
    model_format_version="1",              # model format version
    storage_key="my-data-connection",      # S3 connection details
    metadata={                             # look at "introduction" on types of metadata to capture 
        "int_key": 1,                      
        "bool_key": False,
        "float_key": 3.14,
        "str_key": "str_value",
    }
)

In [ ]:

# add labels
model.custom_properties = {
    "production": "",
    "finance": "",
}
registry.update(model)

# view the model added to the registry
model = registry.get_registered_model("mnist")
print(model)

version = registry.get_model_version("mnist", "1.0.0")
print(version)

## Deploy Model

Switch over into Dashboard into Model Registry screen and find the model inside the dashboard and choose to deploy from the UI. Follow directions [here](./deploy-model.md)